In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    import riskfolio.Portfolio as pf
    riskfolio_available = True
except ImportError:
    riskfolio_available = False


In [ ]:
tickers = {
    "Apple": "AAPL",
    "Tesla": "TSLA",
    "Microsoft": "MSFT",
    "Amazon": "AMZN",
    "NVIDIA": "NVDA"
}

start_date = "2020-01-01"
end_date = "2025-01-01"


In [ ]:
data = {}
for name, ticker in tickers.items():
    print(f"📥 Downloading {name} ({ticker})...")
    stock = yf.download(ticker, start=start_date, end=end_date, group_by="column")

    if not stock.empty:
        if "Adj Close" in stock.columns:
            data[name] = stock["Adj Close"].squeeze()
        elif "Close" in stock.columns:
            print(f"⚠️ {ticker} missing 'Adj Close'. Using 'Close' instead.")
            data[name] = stock["Close"].squeeze()
        else:
            print(f"❌ {ticker} has no valid price columns.")
    else:
        print(f"❌ No data returned for {ticker}")


In [ ]:
if len(data) == 0:
    print("❌ No valid data downloaded.")
else:
    price_df = pd.DataFrame(data).dropna()
    price_df.to_csv("prices.csv")
    display(price_df.tail())


In [ ]:
    returns = price_df.pct_change().dropna()
    corr_matrix = returns.corr()
    plt.figure(figsize=(10, 6))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
    plt.title("🔗 Correlation Matrix of Daily Returns")
    plt.show()


In [ ]:
    if "Apple" in returns.columns and "NVIDIA" in returns.columns:
        roll_corr = returns["Apple"].rolling(60).corr(returns["NVIDIA"])
        roll_corr.plot(title="🔁 60-Day Rolling Correlation: Apple vs NVIDIA", figsize=(10, 4))
        plt.axhline(0, color='gray', linestyle='--')
        plt.grid(True)
        plt.show()


In [ ]:
    if riskfolio_available:
        port = pf.Portfolio(returns=returns)
        port.assets_stats(method_mu='hist', method_cov='hist')
        risk = port.risk_measures(method='hist', rf=0)
        display(risk[["VaR_0.05", "CVaR_0.05", "Sharpe"]].round(4))

        w = port.optimization(model="Classic", rm="MV", obj="Sharpe", hist=True)
        display(w.T.round(4))

        port_weights = w[w > 0].index.tolist()
        weighted_returns = returns[port_weights].mul(w.T[port_weights].values, axis=1).sum(axis=1)
        cumulative_returns = (1 + weighted_returns).cumprod()
        cumulative_returns.plot(title="📈 Optimized Portfolio Cumulative Returns", figsize=(10, 4))
        plt.grid(True)
        plt.show()

        drawdown = (cumulative_returns - cumulative_returns.cummax()) / cumulative_returns.cummax()
        drawdown.plot(title="📉 Drawdown from Peak", figsize=(10, 4), color="red")
        plt.grid(True)
        plt.show()

        rebalance_returns = weighted_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
        rebalance_cumulative = (1 + rebalance_returns).cumprod()
        rebalance_cumulative.plot(title="🔁 Monthly Rebalanced Portfolio", figsize=(10, 4))
        plt.grid(True)
        plt.show()
    else:
        print("⚠️ Install riskfolio-lib to enable risk metrics and optimization.")
